# 50HL30D
![56%](https://progress-bar.dev/56?title=progress&width=100)

#### Description:
- Solving 50 Hard Leetcode problems in 30 days.<br>
- Languange of choice: `Python`<br>

#### Timelines:
- Start Date: 8th Feb 2024
- Challenge End Date: 9th Mar 2024

In [1]:
from typing import List
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
from IPython.display import display

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)
        return visited

    def __str__(self):
        return str(self.val)

    def from_array(nums: List[int]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

temp = BinaryTreeNode.from_array([1,2,3,4,5,6,None,None,None,7,8])
temp.to_list()

[1, 2, 3, 4, 5, 6, None, None, None, 7, 8, None, None, None, None, None, None]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

# 1. 
Date: 08th Feb 2024<br>
Hard Leetcode: reducing-dishes<br>
Solution: https://leetcode.com/problems/reducing-dishes/submissions/1169321043/

In [4]:
class Solution:
    def maxSatisfaction(self, satisfaction: List[int]) -> int:
        # Compute once
        N = len(satisfaction)

        # Sort to find the best possible time for each (order wise)
        satisfaction.sort()

        @functools.cache
        def backtrack(currIdx: int = 0, time: int = 1) -> int:
            if currIdx == N:
                return 0
            else:
                return max(
                    (time * satisfaction[currIdx]) + backtrack(currIdx + 1, time + 1),
                    backtrack(currIdx + 1, time)
                )

        return backtrack()

# Testing the solution
assert Solution().maxSatisfaction([-1, -8, 0, 5, -9]) == 14
assert Solution().maxSatisfaction([4, 3, 2]) == 20
assert Solution().maxSatisfaction([-1, -2, -3]) == 0

# 2. 
Date: 09th Feb 2024<br>
Hard Leetcode: minimum-one-bit-operations-to-make-integers-zero<br>
Solution: https://leetcode.com/problems/minimum-one-bit-operations-to-make-integers-zero/submissions/1170668823/

In [5]:
class Solution:
    def minimumOneBitOperations__(self, n: int) -> int:
        def changeBit(b: str, idx: int):
            bits_ = list(b)
            curr = bits_[idx]
            curr = str(int(not bool(int(curr))))
            bits_[idx] = curr
            return ''.join(bits_)

        def op1(b: str, N: int):
            'Last bit is changed'
            return changeBit(b, N - 1)

        def op2(b: str, N: int):
            """
            Swap ith bit given i-1st bit is 1 and all bits to the right are 0
            """
            i = b.rindex('1')
            if i >= 1:
                return changeBit(b, i - 1)
            else:
                return b

        def backtrack(b: str, processed: set[str] = set()):
            if not int(b):
                return 0
            else:
                op1_result = op1(b, N)
                op2_result = op2(b, N)
                op1_count = op2_count = math.inf
                if op1_result not in processed:
                    op1_count = backtrack(op1_result, {*processed, b})
                if op2_result not in processed:
                    op2_count = backtrack(op2_result, {*processed, b})

                result = 1 + min(op1_count, op2_count)
                return result

        bits = bin(n)[2:]
        N = len(bits)
        return backtrack(bits)

    def minimumOneBitOperations_(self, n: int) -> int:
        if n == 0:
            return 0

        bits = bin(n)[2:]
        bit_length = len(bits)
        maxN = (2 ** bit_length) - 1
        minN = 2 ** (bit_length - 1)
        dp = [_ for _ in range(minN, maxN + 1)]

        def splitAndMerge(arr: list[int], rev=False):
            N = len(arr)
            if N == 1:
                return arr
            else:
                mid = N // 2
                left = splitAndMerge(arr[:mid], rev=False)
                right = splitAndMerge(arr[mid:], rev=True)
                return right + left if rev else left + right

        dp = splitAndMerge(dp, rev=True)
        return dp[n - minN]

    def minimumOneBitOperations(self, n: int) -> int:
        # https://www.youtube.com/watch?v=yRI18_MaG7k
        if n == 0:
            return 0
        else:
            k = 0
            while 2 ** k <= n:
                k += 1
            k -= 1
            return 2 ** (k + 1) - 1 - self.minimumOneBitOperations(2 ** k ^ n)

# Testing the solution
for _ in range(1, 300):
    assert Solution().minimumOneBitOperations_(_) == Solution().minimumOneBitOperations(_), f"failed for {_}"

# 3. 
Date: 10th Feb 2024<br>
Hard Leetcode: human-traffic-of-stadium<br>
Solution: https://leetcode.com/problems/human-traffic-of-stadium/submissions/1171130705

In [6]:
temp = read_lc_df("""
+------+------------+-----------+
| id   | visit_date | people    |
+------+------------+-----------+
| 1    | 2017-01-01 | 10        |
| 2    | 2017-01-02 | 109       |
| 3    | 2017-01-03 | 150       |
| 4    | 2017-01-04 | 99        |
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |
+------+------------+-----------+
""")

temp['visit_date'] = pd.to_datetime(temp['visit_date'])

def human_traffic_df(stadium: pd.DataFrame) -> pd.DataFrame:
    df = stadium[stadium['people'] >= 100]
    ids = set(df['id'])
    result = []
    for row in df.itertuples():
        row = row[1:]
        curr_id = row[0]
        if curr_id - 1 in ids and curr_id - 2 in ids:
            result.append(row)
        elif curr_id - 1 in ids and curr_id + 1 in ids:
            result.append(row)
        elif curr_id + 1 in ids and curr_id + 2 in ids:
            result.append(row)

    df = pd.DataFrame.from_records(result, columns=stadium.columns)
    df.columns = stadium.columns
    return df

# Pandas / python solution
display(human_traffic_df(temp))

# Write to SQL DB
temp.to_sql("temp", if_exists='replace', con=conn, index=False)

display(pd.read_sql("""
    with cte1 as (
        select *,  id - row_number() over(order by id asc) as rnk
        from temp
        where people >= 100
    ),

    cte2 as (
        select c.*, count(*) over (partition by rnk) rnk_count from cte1 c order by c.visit_date
    )

    select c.id, c.visit_date, c.people from cte2 c where c.rnk_count > 2
""", con=conn))

display(pd.read_sql("""
    with cte1 as (
        select * from temp where people >= 100
    ),

    cte2 as (
        select
                c.*,
                ifnull(-c.id - 1 + lead(id, 1) over (), 1) = 0 as lead1,
                ifnull(-c.id - 2 + lead(id, 2) over (), 1) = 0 as lead2,
                ifnull(c.id - 1 - lag(id, 1) over (), 1) = 0 as lag1,
                ifnull(c.id - 2 - lag(id, 2) over (), 1) = 0 as lag2
        from cte1 c
    ),

    cte3 as (
        select
            c.*,
            (lead1 + lead2 + lag1 + lag2) as filter
        from cte2 c
    )

    select c.id, c.visit_date, c.people from cte3 c where c.filter >= 2
""", con=conn))

,id,visit_date,people
0,5.0,2017-01-05,145.0
1,6.0,2017-01-06,1455.0
2,7.0,2017-01-07,199.0
3,8.0,2017-01-09,188.0


,id,visit_date,people
0,5.0,2017-01-05 00:00:00,145.0
1,6.0,2017-01-06 00:00:00,1455.0
2,7.0,2017-01-07 00:00:00,199.0
3,8.0,2017-01-09 00:00:00,188.0


,id,visit_date,people
0,5.0,2017-01-05 00:00:00,145.0
1,6.0,2017-01-06 00:00:00,1455.0
2,7.0,2017-01-07 00:00:00,199.0
3,8.0,2017-01-09 00:00:00,188.0


# 4. 
Date: 10th Feb 2024<br>
Hard Leetcode: largest-rectangle-in-histogram<br>
Solution: https://leetcode.com/problems/largest-rectangle-in-histogram/submissions/1171340811

In [7]:
class Solution:
    def largestRectangleArea(self, heights: List[int]) -> int:
        """
        For each hist, count the number of adjacent hist less than or equal to itself
        Travel left to right, find the next smallest number, repeat by travelling from left to right
        """
        N = len(heights)
        nge_rtl, nge_ltr = [N - 1 for _ in range(N)], [0 for _ in range(N)]
        stack = []
        for i in range(N):
            while stack and heights[stack[-1]] > heights[i]:
                nge_rtl[stack.pop()] = i - 1
            stack.append(i)

        stack = []
        for i in range(N - 1, -1, -1):
            while stack and heights[stack[-1]] > heights[i]:
                nge_ltr[stack.pop()] = i + 1
            stack.append(i)

        maxArea = -math.inf
        for i in range(N):
            curr = heights[i]
            curr = (nge_rtl[i] - nge_ltr[i] + 1) * curr
            maxArea = max(maxArea, curr)

        return maxArea

# Testing the soltuion
assert Solution().largestRectangleArea([2, 1, 5, 6, 2, 3]) == 10
assert Solution().largestRectangleArea([2,4]) == 4

# 5. 
Date: 11th Feb 2024<br>
Hard Leetcode: cherry-pickup-II<br>
Solution: https://leetcode.com/problems/cherry-pickup-ii/submissions/1172053835/

In [8]:
class Solution:
    def cherryPickup(self, grid: List[List[int]]) -> int:
        M = len(grid)
        N = len(grid[0])

        @functools.cache
        def backtrack(i: int, r1: int, r2: int) -> int:
            if 0 <= r1 < N and 0 <= r2 < N and r1 != r2:
                curr = grid[i][r1] + grid[i][r2]
                if i == M - 1:
                    return curr
                else:
                    return curr + max(
                        backtrack(i + 1, r1 - 1, r2 - 1),
                        backtrack(i + 1, r1 - 1, r2),
                        backtrack(i + 1, r1 - 1, r2 + 1),
                        backtrack(i + 1, r1, r2 - 1),
                        backtrack(i + 1, r1, r2),
                        backtrack(i + 1, r1, r2 + 1),
                        backtrack(i + 1, r1 + 1, r2 - 1),
                        backtrack(i + 1, r1 + 1, r2),
                        backtrack(i + 1, r1 + 1, r2 + 1),
                    )
            else:
                return -math.inf

        result = backtrack(0, 0, N - 1)
        return result

# Testing the solution
assert Solution().cherryPickup([[3,1,1],[2,5,1],[1,5,5],[2,1,1]]) == 24
assert Solution().cherryPickup([[1,0,0,0,0,0,1],[2,0,0,0,0,3,0],[2,0,9,0,0,0,0],[0,3,0,5,4,0,0],[1,0,2,3,0,0,6]]) == 28

# 6. 
Date: 11th Feb 2024<br>
Hard Leetcode: candy<br>
Solution: https://leetcode.com/problems/candy/submissions/1172096247

In [9]:
class Solution:
    def candy(self, ratings: List[int]) -> int:
        """
        Initially all positions are Unknown
        Start with the lowest rated indices all the way till the highest rated indices

        1. Both neighbours are unknown - assign the student with 1 child
        2. One neighbour is known - increment neighbour + 1
        3. Both neighbours are known - max(left_neighbour, right_neighbour) + 1

        [2, 2, 0, 3, 1, 0]

        [_, _, _, _, _, _]
        [_, _, 1, _, _, 1]
        [_, _, 1, _, 2, 1]
        [1, 2, 1, _, 2, 1]
        [1, 2, 1, 3, 2, 1]

        sum([1, 2, 1, 3, 2, 1]) => 10
        """

        N = len(ratings)
        indices = dict()
        for i in range(N):
            curr = indices.get(ratings[i], [])
            curr.append(i)
            indices[ratings[i]] = curr

        if N == 1:
            return 1
        else:
            dp = [0 for _ in range(N)]
            for key in sorted(indices.keys()):
                curr = [*dp]
                for idx in indices[key]:
                    dp[idx] = 1 + max(curr[idx - 1] if idx > 0 else -math.inf, curr[idx + 1] if idx < N - 1 else -math.inf)
            return sum(dp)

# Testing the solution
assert Solution().candy([1,0,2]) == 5
assert Solution().candy([0]) == 1
assert Solution().candy([1,2,2]) == 4
assert Solution().candy([1,2,3,3]) == 7
assert Solution().candy([1,2,3,3,1]) == 9
assert Solution().candy([1,2,3,3,1,0]) == 12 
assert Solution().candy([2,2,3,3,1,0]) == 10
assert Solution().candy([2,2,0,3,1,0]) == 10
assert Solution().candy([1,2,0,2,0,0,1,2,2,3,4,0,5]) == 22

# 7. 
Date: 12th Feb 2024<br>
Hard Leetcode: n-queens<br>
Solution: https://leetcode.com/problems/n-queens/submissions/1173381198

In [10]:
class Solution:
    def solveNQueens(self, N: int) -> List[List[str]]:
        board = [["." for _ in range(N)] for __ in range(N)]

        def canPlace(i_: int, j_: int) -> bool:
            """
            This function can be optimized. Instead of iterating through to all the cells - 
            Simply check for each placed queen if the current position is a conflict. Gonna try later :)

            This would make the checks for each queen constant time.
            """
            for dir_ in [(-1, 0), (1, 0), (0, -1), (0, 1), (1, 1), (-1, -1), (-1, 1), (1, -1)]:
                row, col = i_, j_
                while 0 <= row < N and 0 <= col < N:
                    if board[row][col] == 'Q':
                        return False
                    row, col = row + dir_[0], col + dir_[1]
            return True

        result = []
        def backtrack(i: int = 0) -> List[List[str]]:
            if i == N:
                result.append(["".join(row_) for row_ in board])
                return
            for j in range(N):
                if canPlace(i, j):
                    board[i][j] = 'Q'
                    backtrack(i + 1)
                    board[i][j] = '.'

        backtrack()
        return result

# Testing the solution
assert Solution().solveNQueens(4) == [['.Q..', '...Q', 'Q...', '..Q.'], ['..Q.', 'Q...', '...Q', '.Q..']]

# 8. 
Date: 13th Feb 2024<br>
Hard Leetcode: dungeon-game<br>
Solution: https://leetcode.com/problems/dungeon-game/submissions/1173675729

In [11]:
class Solution:
    def calculateMinimumHP(self, dungeon: List[List[int]]) -> int:
        """
        If curr is positive or 0, minHealth is 1
        if curr is negative, 1 - curr

        Keep track of minHealth required to start at that cell and also accumulated health
        """

        M = len(dungeon)
        N = len(dungeon[0])

        for i in range(M - 1, -1, -1):
            for j in range(N - 1, -1, -1):
                curr = 1 - dungeon[i][j]
                safestChoicePath = min(dungeon[i + 1][j] if i + 1 < M else math.inf, dungeon[i][j + 1] if j + 1 < N else math.inf)
                if safestChoicePath == math.inf:
                    safestChoicePath = 1
                dungeon[i][j] = max(1, curr + safestChoicePath - 1)

        return dungeon[0][0]

# Testing the solution
assert Solution().calculateMinimumHP([[-2,-3,3],[-5,-10,1],[10,30,-5]]) == 7
assert Solution().calculateMinimumHP([[2,3,3],[-5,10,1],[10,30,5]]) == 1
assert Solution().calculateMinimumHP([[1,2,3],[-4,-5,6],[-7,-8,-9]]) == 1
assert Solution().calculateMinimumHP([[0]]) == 1

# 9. 
Date: 13th Feb 2024<br>
Hard Leetcode: number-of-digit-one<br>
Solution: https://leetcode.com/problems/number-of-digit-one/submissions/1174159411

In [12]:
class Solution:

    def countDigitOne_(self, n: int) -> int:
        i, prev, curr = 0, 0, 0
        while i <= n:
            curr = prev + str(i).count('1')
            prev = curr
            i += 1
        return curr

    def countDigitOne(self, n: int) -> int:
        """
        How many times does 1 occur for each digit position?

        For eg:
                Say n = 999 (100, 10, 1)
                1 in 100s position: 100
                1 in 10s position: 100
                1 in 1s position: 100

        For max n: n_digits * (10 ^ (n_digits - 1))

        General Logic:
        Iterate through digits (right to left, even left to right should work)
            If curr == 0: result += left * factor
            Elif curr == 1: result += (left * factor) + (right + 1)
            Else: result += (left + 1) * factor

        """

        result, factor = 0, 1
        s = str(n)
        N = len(s)

        for i in range(-1, -N -1, -1):

            curr = int(s[i])
            left = int(s[:i]) if i > -N else -1 if s[0] == 1 else 0
            right = int(s[i+1:]) if i < -1 else -1 if s[-1] == 0 else 0

            if curr == 0:
                result += left * factor
            elif curr == 1:
                result += (left * factor) + right + 1
            else:
                result += (left + 1) * factor

            factor *= 10

        return result

# Testing the solution
for _ in tqdm.tqdm(range(1001)):
    assert Solution().countDigitOne_(_) == Solution().countDigitOne(_), f"Failed for {_}"

  0%|                                                                                                                                               | 0/1001 [00:00<?, ?it/s]

 96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏     | 957/1001 [00:00<00:00, 9560.73it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1001/1001 [00:00<00:00, 9052.64it/s]

# 10. 
Date: 14th Feb 2024<br>
Hard Leetcode: my-calendar-iii<br>
Solution: https://leetcode.com/problems/my-calendar-iii/submissions/1174676799/

In [13]:
class MyCalendarThree:

    def __init__(self):
        self.calendar = []
        self.K = -math.inf

    def book_(self, startTime: int, endTime: int) -> int:
        "Will timeout"
        K = -math.inf
        for ct in range(startTime, endTime + 1):
            curr = 1
            for es, ee in self.calendar:
                if es < ct < ee or es < ct < ee:
                    curr += 1
            K = max(K, curr)

        self.calendar.append([startTime, endTime])
        self.K = max(self.K, K)
        return self.K

    def book_(self, startTime: int, endTime: int) -> int:
        "Optimized a bit by using dictionaries to remove duplicates. Time complexity in worst case is still 2^N"
        updated = dict()
        for es, ee, ec in self.calendar:
            lt, gt = sorted([[startTime, endTime], [es, ee]])
            if lt[0] <= gt[0] < lt[1]:
                overlap = (max(startTime, es), min(endTime, ee))
                updated[overlap] = max(updated.get(overlap, 0), ec + 1)
            updated[(es, ee)] = max(updated.get((es, ee), 0), ec)
        updated[(startTime, endTime)] = max(updated.get((startTime, endTime), 0), 1)
        self.calendar = list(map(lambda x: (x[0][0], x[0][1], x[1]), updated.items()))
        return max(self.calendar, key=lambda x: x[2])[2]

    def book(self, startTime: int, endTime: int) -> int:
        "https://www.youtube.com/watch?v=gbJKlcokAIo"

        start = bisect.bisect(self.calendar, (startTime, 1))
        if start < len(self.calendar) and self.calendar[start][0] == startTime:
            self.calendar[start] = (startTime, self.calendar[start][1] + 1)
        else:
            bisect.insort(self.calendar, (startTime, 1))

        end = bisect.bisect(self.calendar, (endTime, 1))
        if end < len(self.calendar) and self.calendar[end][0] == endTime:
            self.calendar[end] = (endTime, self.calendar[end][1] - 1)
        else:
            bisect.insort(self.calendar, (endTime, -1))

        K = -math.inf
        curr = 0
        for _, c in self.calendar:
            curr += c
            K = max(K, curr)

        return K

# Testing the solution
Solution = MyCalendarThree()
temp = (
    [[10, 20], [50, 60], [10, 40], [5, 15], [5, 10], [25, 55], [0, 100]],
    [1, 1, 2, 3, 3, 3, 4]
)
for i in range(len(temp[0])):
    assert Solution.book(*temp[0][i]) == temp[1][i]

# 11. 
Date: 15th Feb 2024<br>
Hard Leetcode: sliding-window-maximum<br>
Solution: https://leetcode.com/problems/sliding-window-maximum/submissions/1175614381

In [14]:
class Solution:
    def maxSlidingWindow_(self, nums: List[int], k: int) -> List[int]:
        "Times out"
        N = len(nums)
        if k == 1:
            return nums
        elif N == k:
            return [max(nums)]
        else:
            stack = sorted(((n, i) for i, n in enumerate(nums)))
            result = [None] * N
            for n, start in stack:
                result[start: start + k] = [n] * k
            result = result[k - 1: 1 - k]
            return result

    def maxSlidingWindow(self, nums: List[int], k: int) -> List[int]:
        """
        https://www.youtube.com/watch?v=CZQGRp93K4k
        """
        N = len(nums)
        deq = collections.deque()
        result = []
        for i in range(N):
            while deq and nums[deq[-1]] < nums[i]:
                deq.pop()
            deq.append(i)
            while deq[0] <= i - k:
                deq.popleft()
            if i >= k - 1:
                result.append(nums[deq[0]])
        return result

# Testing the solution
assert Solution().maxSlidingWindow([9, 8, 7, 6, 5], 5) == [9]
assert Solution().maxSlidingWindow([9, 8, 7, 6, 5], 1) == [9, 8, 7, 6, 5]
assert Solution().maxSlidingWindow([9, 8, 7, 6, 5, 4, 3, 2, 1], 3) == [9, 8, 7, 6, 5, 4, 3]
assert Solution().maxSlidingWindow([1, 3, -1, -3, 5, 3, 6, 7], 3) == [3, 3, 5, 5, 6, 7]
assert Solution().maxSlidingWindow([1, 3, -1, 3, 5, 9, 6, 7, 1], 3) == [3, 3, 5, 9, 9, 9, 7]
assert Solution().maxSlidingWindow([1, 3, -1, 3, 5, 9, 6, 7], 5) == [5, 9, 9, 9]
assert Solution().maxSlidingWindow([-7, -8, 7, 5, 7, 1, 6, 0],  4) == [7, 7, 7, 7, 7]

# 12. 
Date: 16th Feb 2024<br>
Hard Leetcode: word-search-ii<br>
Solution: https://leetcode.com/problems/word-search-ii/submissions/1176699366

In [15]:
class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:

        def prune_prefix_tree(word: str, curr_prefix: dict) -> None:
            if word == "":
                del curr_prefix["*"]
            else:
                prune_prefix_tree(word[1:], curr_prefix[word[0]])
                if len(curr_prefix) == 0:
                    del curr_prefix[word[0]]

        result = set()
        def backtrack(i: int, j: int, curr_prefix: dict, visited: set[tuple[int]]):
            if 0 <= i < M and 0 <= j < N and board[i][j] in curr_prefix:
                next_prefix = curr_prefix[board[i][j]]
                if "*" in next_prefix:
                    result.add(next_prefix["*"])
                    prune_prefix_tree(next_prefix["*"], prefix_tree)
                for dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    i_, j_ = i + dir[0], j + dir[1]
                    if (i_, j_) not in visited:
                        backtrack(i_, j_, next_prefix, {*visited, (i_, j_)})

        # Construct the prefix tree: O(word_length * number of words)
        prefix_tree = dict()
        for word in words:
            curr = prefix_tree
            for ch in word + "*":
                curr[ch] = curr.get(ch, dict())
                if ch == "*":
                    curr[ch] = word
                curr = curr[ch]

        # Backtrack into the grid as long as there is a match, if match - remove word 
        M = len(board)
        N = len(board[0])
        for i in range(M):
            for j in range(N):
                backtrack(i, j, prefix_tree, {(i, j)})

        return result

# Testing the solution
Solution().findWords([["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]], ["oath","pea","eat","rain", "oatak"])

{'eat', 'oatak', 'oath'}

# 13. 
Date: 16th Feb 2024<br>
Hard Leetcode: maximum-score-words-formed-by-letters<br>
Solution: https://leetcode.com/problems/maximum-score-words-formed-by-letters/submissions/1177035354/

In [16]:
class Solution:

    def sub_dict(d1: dict, d2: dict):
        updated = dict(d1)
        for ch in d2:
            if ch not in updated or updated[ch] < d2[ch]:
                return None
            else:
                updated[ch] -= d2[ch]
        return updated

    def maxScoreWords(self, words: List[str], letters: List[str], score: List[int]) -> int:

        N_WORDS = len(words)

        # Letters_dict, words_dict, scores_dict
        letters_ = dict(collections.Counter(letters))
        words_ = []
        scores_ = []
        for word in words:
            word_ = dict()
            score_ = 0
            for ch in word:
                word_[ch] = word_.get(ch, 0) + 1
                score_ += score[ord(ch) - 97]
            words_.append(word_)
            scores_.append(score_)

        def backtrack(idx: int, ld: dict) -> int:

            if idx >= N_WORDS:
                return 0

            word_freq = words_[idx]
            result = -math.inf

            # Pick
            ld_pick = Solution.sub_dict(ld, word_freq)
            if ld_pick:
                result = scores_[idx] + backtrack(idx + 1, ld_pick)

            # No Pick
            result = max(result, backtrack(idx + 1, ld))

            return result 

        result = backtrack(0, letters_)
        return result

# Testing the solution
assert Solution().maxScoreWords(["dog","cat","dad","good"], ["a","a","c","d","d","d","g","o","o"], [1,0,9,5,0,0,3,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0]) == 23
assert Solution().maxScoreWords(["xxxz","ax","bx","cx"], ["z","a","b","c","x","x","x"], [4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,10]) == 27
assert Solution().maxScoreWords(["leetcode"], ["l","e","t","c","o","d"], [0,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0]) == 0
assert Solution().maxScoreWords(["baa","bba","ccb","ac"], ["a","b","b","b","b","c"], [2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]) == 6

# 14. 
Date: 17th Feb 2024<br>
Hard Leetcode: permutation-sequence<br>
Solution: https://leetcode.com/problems/permutation-sequence/submissions/1177409098/

In [17]:
class Solution:
    def getPermutation(self, n: int, k: int) -> str:

        total = math.factorial(n)
        nums = [str(_) for _ in range(1, n + 1)]

        def backtrack(nums_: list[int], k_: int, n_: int, i_: int) -> str:
            if len(nums_) == 1:
                return nums_[0]

            curr_idx = k_ // n_
            curr = nums_[k_ // n_]

            return curr + backtrack(sorted(nums_[:curr_idx] + nums_[curr_idx + 1:]), k_ % n_, n_ // i_, i_ - 1)

        return backtrack(nums, k - 1, total // n, n - 1)

    def naive(self, n: int, k: int) -> str:
        nums = [str(_) for _ in range(1, n + 1)]
        perms = itertools.permutations(nums, n)
        result = None
        for _ in range(k):
            result = next(perms)

        return "".join(result)

# Testing the Solution
for n in tqdm.tqdm(range(1, 8)):
    for k in range(1, math.factorial(n) + 1):
        assert Solution().naive(n, k) == Solution().getPermutation(n, k)

assert Solution().getPermutation(9, math.factorial(9)) == '987654321'

  0%|                                                                                                                                                  | 0/7 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  6.08it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  6.08it/s]

# 15. 
Date: 17th Feb 2024<br>
Hard Leetcode: max-points-on-a-line<br>
Solution: https://leetcode.com/problems/max-points-on-a-line/submissions/1177463531/

In [18]:
class Solution:
    def equation(pt1: List[int], pt2: List[int]) -> List[int]:
        """
        https://math.stackexchange.com/a/1734022
        """

        # Points X, Y
        (x1, y1), (x2, y2) = pt1, pt2

        # Y = MX + C
        if abs(x2 - x1) > 0:
            M = (y2 - y1) / (x2 - x1)
            C = ((x2 * y1) - (x1 * y2)) / (x2 - x1)
            return M, C, None

        else: # For cases where X = C
            return None, None, x1

    def maxPoints(self, points: List[List[int]]) -> int:
        N = len(points)
        if N == 1:
            return 1
        else:
            hm: dict[tuple[int], set[tuple[int]]] = dict()
            for i in range(N - 1):
                for j in range(i + 1, N):
                    eq = Solution.equation(points[i], points[j])
                    line_pts = hm.get(eq, set())
                    line_pts.add(tuple(points[i]))
                    line_pts.add(tuple(points[j]))
                    hm[eq] = line_pts

            return max(map(len, hm.values()))

# Testing the solution
assert Solution().maxPoints([[1, 1], [3, 2], [5, 3], [4, 1], [2, 3], [1, 4], [6, -1]]) == 5
assert Solution().maxPoints([[1, 1], [2, 2], [3, 3], [0, 0]]) == 4
assert Solution().maxPoints([[0, 0]]) == 1
assert Solution().maxPoints([[0, 1], [0, 2], [2, 2], [1, 0], [2, 0]]) == 2

# 16. 
Date: 18th Feb 2024<br>
Hard Leetcode: earliest-possible-day-of-full-bloom<br>
Solution: https://leetcode.com/problems/earliest-possible-day-of-full-bloom/submissions/1178490688/

In [19]:
class Solution:
    def earliestFullBloom(self, plantTime: List[int], growTime: List[int]) -> int:
        """
        The idea is to start with planting the seeds that take the most time
        Calculate remaining time and return the sum
        """
        plantTime, growTime = zip(*sorted(zip(plantTime, growTime), key=lambda x: x[1], reverse=True))

        total = 0
        remaining = None
        for p, g in zip(plantTime, growTime):
            total += p
            remaining = g if remaining is None else max(remaining - p, g)

        return total + remaining

# Test the solution
assert Solution().earliestFullBloom([1, 2, 3, 2],  [2, 1, 2, 1]) == 9
assert Solution().earliestFullBloom([1, 4, 3], [2, 3, 1]) == 9
assert Solution().earliestFullBloom(
    [27, 5, 24, 17, 27, 4, 23, 16, 6, 26, 13, 17, 21, 3, 9, 10, 28, 26, 4, 10, 28, 2],  
    [26, 9, 14, 17, 6, 14, 23, 24, 11, 6, 27, 14, 13, 1, 15, 5, 12, 15, 23, 27, 28, 12]
) == 348

# 17. 
Date: 18th Feb 2024<br>
Hard Leetcode: number-of-visible-people-in-a-queue<br>
Solution: https://leetcode.com/problems/number-of-visible-people-in-a-queue/submissions/1178697347/

In [20]:
class Solution:
    def canSeePersonsCount(self, heights: List[int]) -> List[int]:
        N = len(heights)
        stack = []
        result = []
        for i in range(N - 1, -1, -1):
            count = 1
            while stack and heights[i] > heights[stack[-1]]:
                stack.pop()
                count += 1
            if not stack:
                count -= 1
            result.append(count)
            stack.append(i)

        result.reverse()
        return result

# Testing the solution - the ques was simple,
# but really annoyed me with so many errors
temp = [
    [[1,2,3,4], [1,1,1,0]], [[1,2,4,3], [1,1,1,0]], [[1,3,2,4], [1,2,1,0]],
    [[1,3,4,2], [1,1,1,0]], [[1,4,2,3], [1,2,1,0]], [[1,4,3,2], [1,1,1,0]],
    [[2,1,3,4], [2,1,1,0]], [[2,1,4,3], [2,1,1,0]], [[2,3,1,4], [1,2,1,0]],
    [[2,3,4,1], [1,1,1,0]], [[2,4,1,3], [1,2,1,0]], [[2,4,3,1], [1,1,1,0]],
    [[3,1,2,4], [3,1,1,0]], [[3,1,4,2], [2,1,1,0]], [[3,2,1,4], [2,2,1,0]],
    [[3,2,4,1], [2,1,1,0]], [[3,4,1,2], [1,2,1,0]], [[3,4,2,1], [1,1,1,0]],
    [[4,1,2,3], [3,1,1,0]], [[4,1,3,2], [2,1,1,0]], [[4,2,1,3], [2,2,1,0]],
    [[4,2,3,1], [2,1,1,0]], [[4,3,1,2], [1,2,1,0]], [[4,3,2,1], [1,1,1,0]],
]

for q, a in temp:
    assert Solution().canSeePersonsCount(q) == a

# 18. 
Date: 19th Feb 2024<br>
Hard Leetcode: minimum-insertion-steps-to-make-a-string-palindrome<br>
Solution: https://leetcode.com/problems/minimum-insertion-steps-to-make-a-string-palindrome/submissions/1179413048

In [21]:
class Solution:
    def minInsertions(self, s: str) -> int:

        @functools.cache
        def countInsertionSteps(left: int, right: int) -> int:
            if left < 0 and right >= N:
                return 0
            elif left < 0:
                return N - right
            elif right >= N:
                return left + 1
            elif s[left] == s[right]:
                return countInsertionSteps(left - 1, right + 1)
            else:
                return min(
                    2 + countInsertionSteps(left - 1, right + 1),
                    1 + countInsertionSteps(left - 1, right),
                    1 + countInsertionSteps(left, right + 1),
                )

        N = len(s)
        minSteps = +math.inf
        for i in range(N):
            left, right = i - 1, i + 1
            minSteps = min(minSteps, countInsertionSteps(left, right))
            if i != N - 1:
                left, right = i, i + 1
                minSteps = min(minSteps, countInsertionSteps(left, right))
        return minSteps

# Testing the solution
assert Solution().minInsertions("zzazz") == 0
assert Solution().minInsertions("mbadm") == 2
assert Solution().minInsertions("leetcode") == 5
assert Solution().minInsertions("missouri") == 4

# 19. 
Date: 20th Feb 2024<br>
Hard Leetcode: parsing-a-boolean-expression<br>
Solution: https://leetcode.com/problems/parsing-a-boolean-expression/submissions/1181172767

In [22]:
class Solution:
    def parseBoolExpr(self, expression: str) -> bool:
        stack: list  = []
        for ch in expression:
            if ch not in (")", ","):
                stack.append(ch)
            elif ch != ',':
                expr = []
                while stack[-1] != '(':
                    curr = stack.pop()
                    expr.append(True if curr in ('t', True) else False)
                stack.pop()
                op = stack.pop()
                if op == "!":
                    result = not expr[0]
                elif op == "|":
                    result = any(expr + [False])
                else:
                    result = all(expr + [True])

                stack.append(result)

        return stack[0]

# Testing the Solution
assert Solution().parseBoolExpr("&(|(f))") == False
assert Solution().parseBoolExpr("&(!(f))") == True
assert Solution().parseBoolExpr("|(!(f))") == True
assert Solution().parseBoolExpr("|(f,f,f,t)") == True
assert Solution().parseBoolExpr("|(f,f,f,&(t,t,!(f),t))") == True
assert Solution().parseBoolExpr("!(&(f,t))") == True
assert Solution().parseBoolExpr("!(!(t))") == True

# 20. 
Date: 21th Feb 2024<br>
Hard Leetcode: freedom-trail<br>
Solution: https://leetcode.com/problems/freedom-trail/submissions/1182130541/

In [23]:
class Solution:

    def findRotateSteps(self, ring: str, key: str) -> int:

        def countNextSteps(nextChar: str, currIdx: int) -> list[tuple[int, int]]:
            next_indices = positions[nextChar]
            result: list[tuple[int, int]] = []
            for nextIdx in next_indices:
                steps = abs(currIdx - nextIdx)
                steps = min(steps, ringLength - steps)
                result.append((nextIdx, steps))
            return result

        @functools.cache
        def backtrack(key: str, curr: int):

            if key == "":
                return 0

            ch = key[0]
            minStep = math.inf
            for nextIdx, steps in countNextSteps(ch, curr):
                minStep = min(minStep, 1 + steps + backtrack(key[1:], nextIdx))

            return minStep

        # Compute the length
        ringLength, keyLength = len(ring), len(key)

        # Compute the positions of characters for quicker navigation
        positions: dict[str, list[int]]  = dict()
        for i in range(ringLength):
            ch = ring[i]
            currIndices = positions.get(ch, [])
            currIndices.append(i)
            positions[ch] = currIndices

        result = backtrack(key, 0)
        return result

# Testing the Solution
assert Solution().findRotateSteps("godding", "gd") == 4
assert Solution().findRotateSteps("godding", "godding") == 13
assert Solution().findRotateSteps("sasdaasdsadwqaasadw", "sadwq") == 16

# 21. 
Date: 22nd Feb 2024<br>
Hard Leetcode: stone-game-iv/<br>
Solution: https://leetcode.com/problems/stone-game-iv/submissions/1182531823/

In [24]:
class Solution:
    @functools.cache
    def winnerSquareGame_(self, n: int) -> bool:
        sqrt = math.sqrt(n)
        if n == 0:
            return False
        elif int(sqrt) == sqrt:
            return True
        else:
            start = int(sqrt)
            while start > 0:
                if not self.winnerSquareGame_(n - start * start):
                    return True
                start -= 1
            return False

    def winnerSquareGame(self, n: int) -> bool:
        dp = [False] * (n + 1)
        for curr in range(1, n + 1):
            for i in range(1, int(math.sqrt(curr)) + 1):
                if not dp[curr - i * i]:
                    dp[curr] = True
                    break

        return dp[-1]

assert Solution().winnerSquareGame(1) == True
assert Solution().winnerSquareGame(2) == False
assert Solution().winnerSquareGame(3) == True
assert Solution().winnerSquareGame(4) == True
assert Solution().winnerSquareGame(5) == False

# 22. 
Date: 22nd Feb 2024<br>
Hard Leetcode: strange-printer-ii<br>
Solution: https://leetcode.com/problems/strange-printer-ii/submissions/1183170079/

In [25]:
class Solution:
    def isPrintable(self, targetGrid: List[List[int]]) -> bool:

        def hasCycle(curr: int, visited: set[int]) -> bool:
            if curr not in order:
                return False
            for next_ in order[curr]:
                if next_ in visited:
                    return True
                elif hasCycle(next_, {*visited, curr}):
                    return True

            return False

        M = len(targetGrid)
        N = len(targetGrid[0])

        # For each number, store the top, left, bottom, right
        rectangles: dict[int, tuple[int, int, int, int]] = dict()
        for i in range(M):
            for j in range(N):
                curr = targetGrid[i][j]
                top, left, bottom, right = rectangles.get(curr, (M, N, 0, 0))
                if i < top:
                    top = i
                if j < left:
                    left = j
                if i > bottom:
                    bottom = i
                if j > right:
                    right = j

                rectangles[curr] = top, left, bottom, right

        # Compare each number pairs by checking the overlap
        # If overlap present, ensure that all the cells of the 
        # overlapping grid are of any one of the either number
        # Note that it is okay if the overlapping grid contains 
        # a different number other than the two

        order: dict[int, list[int]] = dict()
        for n1, n2 in itertools.combinations(rectangles, r=2):
            top1, left1, bottom1, right1 = rectangles[n1]
            top2, left2, bottom2, right2 = rectangles[n2]
            top, left, bottom, right = max(top1, top2), max(left1, left2), min(bottom1, bottom2), min(right1, right2)
            overlapping_paints = set()
            for i in range(top, bottom + 1):
                for j in range(left, right + 1):
                    overlapping_paints.add(targetGrid[i][j])

            assert not len(overlapping_paints) or (n1 in overlapping_paints or n2 in overlapping_paints)

            if n1 in overlapping_paints and n2 in overlapping_paints:
                return False
            else:
                if n1 in overlapping_paints:
                    next_ = order.get(n2, [])
                    next_.append(n1)
                    order[n2] = next_
                elif n2 in overlapping_paints:
                    next_ = order.get(n1, [])
                    next_.append(n2)
                    order[n1] = next_
        else:
            # Ensure that there are no cycles
            for start in order:
                if hasCycle(start, set()):
                    return False
            return True

# Testing the solution
assert Solution().isPrintable([[1,1,1,1],[1,1,3,3],[1,1,3,4],[5,5,1,4]]) == True
assert Solution().isPrintable([[1,1,1,1], [1,2,2,1], [1,2,2,1], [1,1,1,1]]) == True
assert Solution().isPrintable([[1,2,1], [2,1,2], [1,2,1]]) == False
assert Solution().isPrintable([[5,1,5,3,5],[4,4,4,3,4],[5,1,5,3,5],[2,1,2,2,2],[5,1,5,3,5]]) == False

# 23. 
Date: 23rd Feb 2024<br>
Hard Leetcode: sudoku-solver<br>
Solution: https://leetcode.com/problems/sudoku-solver/submissions/1184011837

In [26]:
class Solution:
    def solveSudoku(self, board: List[List[str]]) -> None:
        def getPossibilities(i: int, j: int) -> set[str]:
            result: set[str] = set(map(str, range(1, 10)))

            # Eliminate all values already present in same row
            for k in range(9):
                if board[i][k] in result:
                    result.remove(board[i][k])

            # Eliminate all values already present in the same col
            for k in range(9):
                if board[k][j] in result:
                    result.remove(board[k][j])

            # Eliminate all values already present in the same grid
            floor_i = math.floor(i / 3)
            for i_ in range(floor_i * 3, (floor_i + 1) * 3):
                floor_j = math.floor(j / 3)
                for j_ in range(floor_j * 3, (floor_j + 1) * 3):
                    if board[i_][j_] in result:
                        result.remove(board[i_][j_])

            return result

        def backtrack() -> bool:
            missing = 0
            for i in range(9):
                for j in range(9):
                    if board[i][j] == ".":
                        missing += 1
                        for n in getPossibilities(i, j):
                            board[i][j] = n
                            if backtrack():
                                return True
                        else:
                            board[i][j] = "."
                            return False

            return missing == 0

        # Call helper function
        backtrack()


# Testing the solution
temp = [
    ["5","3",".",".","7",".",".",".","."],
    ["6",".",".","1","9","5",".",".","."],
    [".","9","8",".",".",".",".","6","."],
    ["8",".",".",".","6",".",".",".","3"],
    ["4",".",".","8",".","3",".",".","1"],
    ["7",".",".",".","2",".",".",".","6"],
    [".","6",".",".",".",".","2","8","."],
    [".",".",".","4","1","9",".",".","5"],
    [".",".",".",".","8",".",".","7","9"]
]
Solution().solveSudoku(temp)
assert temp == [
        ["5","3","4","6","7","8","9","1","2"],
        ["6","7","2","1","9","5","3","4","8"],
        ["1","9","8","3","4","2","5","6","7"],
        ["8","5","9","7","6","1","4","2","3"],
        ["4","2","6","8","5","3","7","9","1"],
        ["7","1","3","9","2","4","8","5","6"],
        ["9","6","1","5","3","7","2","8","4"],
        ["2","8","7","4","1","9","6","3","5"],
        ["3","4","5","2","8","6","1","7","9"]
]

# 24. 
Date: 24th Feb 2024<br>
Hard Leetcode: find-all-people-with-secret<br>
Solution: https://leetcode.com/problems/find-all-people-with-secret/submissions/1184653107

In [27]:
class Solution:
    def findAllPeople(self, n: int, meetings: List[List[int]], firstPerson: int) -> set[int]:

        calender: dict[int, dict[int, int]] = dict()
        meetings_dict: dict[int, list[set[int]]] = dict()
        for x, y, t in meetings:
            if x in calender and t in calender[x] and y in calender and t in calender[y] and calender[x] != calender[y]:
                room1 = calender[x][t]
                room2 = calender[y][t]
                new_room = set.union(meetings_dict[t][room1], meetings_dict[t][room2])
                meetings_dict[t].append(new_room)
                for p in new_room:
                    calender[p][t] = len(meetings_dict[t]) - 1
            elif x in calender and t in calender[x]:
                room_number = calender[x][t]
                meetings_dict[t][room_number].add(y)
                y_cal = calender.get(y, dict())
                y_cal[t] = room_number
                calender[y] = y_cal
            elif y in calender and t in calender[y]:
                room_number = calender[y][t]
                meetings_dict[t][room_number].add(x)
                x_cal = calender.get(x, dict())
                x_cal[t] = room_number
                calender[x] = x_cal
            else:
                curr = meetings_dict.get(t, [])
                curr.append({x, y})
                meetings_dict[t] = curr
                y_cal = calender.get(y, dict())
                y_cal[t] = len(curr) - 1
                x_cal = calender.get(x, dict())
                x_cal[t] = len(curr) - 1
                calender[y] = y_cal
                calender[x] = x_cal

        rooms_visited: set[tuple[int, int]] = set()
        secrets_known_to: dict[int, int] = {firstPerson: 0, 0: 0}
        def share_secret(person: int, time: int) -> None:
                for t in calender.get(person, []):
                    if t >= time:
                        room = calender[person][t]
                        if (t, room) not in rooms_visited:
                            rooms_visited.add((t, room))
                            for p in meetings_dict[t][room]:
                                if p not in secrets_known_to or t < secrets_known_to[p]:
                                    secrets_known_to[p] = t
                                    share_secret(p, t)

        share_secret(0, 0)
        share_secret(firstPerson, 0)

        return set(secrets_known_to.keys())

# Testing the solution
assert Solution().findAllPeople(6, [[1,2,5],[2,3,8],[1,5,10]], 1) == {0,1,2,3,5}
assert Solution().findAllPeople(4, [[3,1,3],[1,2,2],[0,3,3]], 3) == {0,1,3}
assert Solution().findAllPeople(5, [[3,4,2],[1,2,1],[2,3,1]], 1) == {0,1,2,3,4}
assert Solution().findAllPeople(10, [[1,2,3],[1,0,1],[9,2,1],[5,4,3],[6,7,9]], 7) == {0, 1, 2, 6, 7}
assert Solution().findAllPeople(7, [[6,2,1],[3,4,1],[4,5,1],[1,0,2],[3,2,1]], 2) == {0, 1, 2, 3, 4, 5, 6}

# 25. 
Date: 25th Feb 2024<br>
Hard Leetcode: minimum-number-of-flips-to-convert-binary-matrix-to-zero-matrix<br>
Solution: https://leetcode.com/problems/minimum-number-of-flips-to-convert-binary-matrix-to-zero-matrix/submissions/1185271529/

In [28]:
class Solution:
    @staticmethod
    def flip(matrix: tuple[tuple[int]], i: int, j: int, M: int, N: int) -> tuple[tuple[int]]:
        result = [[cell for cell in row] for row in matrix]
        for i_, j_ in ((i, j), (i - 1, j), (i, j - 1), (i + 1, j), (i, j + 1)):
            if 0 <= i_ < M and 0 <= j_ < N:
                result[i_][j_] = int(not bool(result[i_][j_]))
        return tuple(map(tuple, result))

    def minFlips(self, mat: List[List[int]]) -> int:
        M = len(mat)
        N = len(mat[0])

        start = tuple([tuple([cell for cell in row]) for row in mat])
        cache: dict[tuple[tuple[int]], int] = dict()

        def backtrack(matrix: tuple[tuple[int]], visited: set[tuple[tuple[int]]] = set()):
            if any([matrix[_][__] for _ in range(M) for __ in range(N)]) == False:
                return 0
            elif matrix in cache:
                return cache[matrix]
            elif matrix in visited:
                return math.inf
            else:
                minSteps = math.inf
                for i in range(M):
                    for j in range(N):
                        next_matrix = Solution.flip(matrix, i, j, M, N)
                        minSteps = min(minSteps, 1 + backtrack(next_matrix, {*visited, matrix}))

                cache[matrix] = minSteps
                return minSteps

        result = backtrack(start)
        return result if result != math.inf else -1

# Testing the solution
assert Solution().minFlips([[0,0],[0,1]]) == 3
assert Solution().minFlips([[0]]) == 0
assert Solution().minFlips([[1,0,0],[1,0,0]]) == -1

# 26. 
Date: 25th Feb 2024<br>
Hard Leetcode: couples-holding-hands<br>
Solution: https://leetcode.com/problems/couples-holding-hands/submissions/1185667325/

In [29]:
class Solution:
    @staticmethod
    def getPair(m: int) -> int:
        return m + 1 if m % 2 == 0 else m - 1

    def minSwapsCouples(self, row: List[int]) -> int:

        N = len(row)
        seated_at = {n: i for i, n in enumerate(row)}

        count = 0
        for i in range(N // 2):
            seat_a, seat_b = i * 2, (i * 2) + 1
            a, b = row[seat_a], row[seat_b]
            a_match, b_match = Solution.getPair(a), Solution.getPair(b)
            a_match_seat, b_match_seat = seated_at[a_match], seated_at[b_match]
            if a_match != b:
                row[seat_b], row[a_match_seat] = row[a_match_seat], row[seat_b]
                seated_at[a_match] = seat_b
                seated_at[b] = a_match_seat
                count += 1

        return count

# Testing the solution
assert Solution().minSwapsCouples([0,2,1,3]) == 1
assert Solution().minSwapsCouples([3,2,0,1]) == 0

# 27. 
Date: 26th Feb 2024<br>
Hard Leetcode: maximize-items<br>
Solution: https://leetcode.com/problems/maximize-items/submissions/1186929885

In [30]:
temp = read_lc_df("""
+---------+----------------+---------------+----------------+
| item_id | item_type      | item_category | square_footage |
+---------+----------------+---------------+----------------+
| 1374    | prime_eligible | Watches       | 68.00          |
| 4245    | not_prime      | Art           | 26.40          |
| 5743    | prime_eligible | Software      | 325.00         |
| 8543    | not_prime      | Clothing      | 64.50          |
| 2556    | not_prime      | Shoes         | 15.00          |
| 2452    | prime_eligible | Scientific    | 85.00          |
| 3255    | not_prime      | Furniture     | 22.60          |
| 1672    | prime_eligible | Beauty        | 8.50           |
| 4256    | prime_eligible | Furniture     | 55.50          |
| 6325    | prime_eligible | Food          | 13.20          |
+---------+----------------+---------------+----------------+
""")

def maximize_items(inventory: pd.DataFrame) -> pd.DataFrame:

    total = 500_000
    agg: dict = inventory.groupby('item_type')['square_footage'].agg(['sum', 'count']).to_dict(orient='index')

    prime_count = (total // agg['prime_eligible']['sum']) if 'prime_eligible' in agg else 0
    total -= (prime_count * agg['prime_eligible']['sum']) if 'prime_eligible' in agg else 0
    non_prime_count = total // agg['not_prime']['sum'] if 'not_prime' in agg else 0

    if 'prime_eligible' in agg:
        result = [
            ['prime_eligible', int(prime_count * agg['prime_eligible']['count'])],
        ]
        if 'not_prime' in agg:
            result.append(['not_prime', int(non_prime_count * agg['not_prime']['count'])])
    else:
        result = [['not_prime', None]]

    result_df: pd.DataFrame = pd.DataFrame(result, columns=['item_type', 'item_count'])
    return result_df

# Print the pandas solution
display(maximize_items(temp))

# Write to SQLite
temp.to_sql("temp", con=conn, index=False, if_exists='replace')

# Execute SQL query and display result
pd.read_sql("""
    with agg as (
        select
            item_type,
            sum(square_footage) total_sq,
            count(item_id) cnt
        from temp
        group by item_type
    ),

    prime as (
        select
            'prime_eligible' item_type,
            CASE
                WHEN NOT EXISTS (select * from agg where item_type = 'prime_eligible') THEN NULL
                ELSE CAST(500000 / (select total_sq from agg where item_type = 'prime_eligible') AS INT)
            END item_count
    ),

    not_prime as (
        select
            'not_prime' item_type,
            CASE
                WHEN NOT EXISTS (select * from agg where item_type = 'not_prime') THEN NULL
                ELSE (
                    CAST((500000 - ((select item_count from prime) * (select total_sq from agg where item_type = 'prime_eligible'))) /
                    (select total_sq from agg where item_type = 'not_prime') as INT)
                )
            END item_count
    ),

    result_intermediate as (

        select
            item_type,
            item_count * (select cnt from agg where item_type='prime_eligible') item_count
        from prime
        where item_count is not NULL

        UNION

        select
            item_type,
            item_count * (select cnt from agg where item_type='not_prime') item_count
        from not_prime
        where item_count is not NULL
    ),

    result as (
        select * from result_intermediate
        union
        select
            'non_prime' item_type,
            NULL item_count
        where not exists(
            select * from result_intermediate
        )
    )

    select * from result

""", con=conn)

,item_type,item_count
0,prime_eligible,5400
1,not_prime,8


,item_type,item_count
0,not_prime,8
1,prime_eligible,5400


# 28. 
Date: 29th Feb 2024<br>
Hard Leetcode: number-of-atoms<br>
Solution: https://leetcode.com/problems/number-of-atoms/submissions/1189837478/

In [31]:
class Solution:
    def countOfAtoms(self, formula: str) -> str:
        """
        Curr could be one of: '(', ')', 'A-Z', 'a-z', '0-9', '<END>'
        """

        N = len(formula)
        idx = 0

        # Step 1 - Group the forumlae & nos together for easy processing
        formula_list: list = []
        for i in range(N):
            ch = formula[i]
            if ch.islower() or (ch.isdigit() and formula_list[-1].isdigit()):
                formula_list[-1] = formula_list[-1] + ch
            else:
                formula_list.append(ch)

        # Recompute N
        N = len(formula_list)

        # Step 2 - For each element, convert it into (ch, count)
        for i in range(N):
            if formula_list[i].isalpha():
                formula_list[i] = [formula_list[i], 1]

        # Step 3 - For each count, if prev is a list - multiply with list[1], 
        # else if it is a () multiply the number inside all of the element pairs
        updated: list = []
        for i in range(N):
            curr = formula_list[i]
            if type(curr) == str and curr.isdigit():
                if type(updated[-1]) == list:
                    prev = updated[-1]
                    updated[-1] = [prev[0], prev[1] * int(curr)]
                elif updated[-1] == ')':
                    nested = 1
                    idx = len(updated) - 2
                    while nested > 0:
                        if updated[idx] == ')':
                            nested += 1
                        elif updated[idx] == '(':
                            nested -= 1
                        else:
                            prev = updated[idx]
                            updated[idx] = [prev[0], prev[1] * int(curr)]
                        idx -= 1
            else:
                updated.append(curr)

        # Each element may appear many times, create a dictionary and continue counting for each 
        counter: dict[str, int] = dict()
        for curr in updated:
            if type(curr) == list:
                counter[curr[0]] = counter.get(curr[0], 0) + curr[1]

        # Sort it based on the alphbetical ordering of elements
        counter_sorted: list[list] = sorted(counter.items(), key=lambda x: x[0])

        # Only show numbers when count is greater than 1
        result = "".join(map(lambda x: f"{x[0]}{x[1] if x[1] > 1 else ''}", counter_sorted))
        return result

# Testing the solution
assert Solution().countOfAtoms("H2O") == "H2O"
assert Solution().countOfAtoms("Mg(OH)2") == "H2MgO2"
assert Solution().countOfAtoms("K4(ON(SO3)2)2") == "K4N2O14S4"
assert Solution().countOfAtoms("(MgOH)") == "HMgO"